In [1]:
import requests
import pandas as pd
import numpy as np
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
r = requests.get(url)
json = r.json()
json.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

In [2]:
%run soup.ipynb #loading dataframes from fbref.com

pl_adv_gk loaded!
pl_std loaded!
pl_shot loaded!
pl_pos loaded!
pl_time loaded!
pl_pass loaded!
pl_pass_type loaded!
pl_msc loaded!
pl_gk loaded!
pl_goal loaded!


In [3]:
elements_df = pd.DataFrame(json['elements'])
teams_df = pd.DataFrame(json['teams'])

In [4]:
def del_id_pl(df):#removing identical players from dataframes
    df.sort_values('player',inplace=True)
    df.reset_index(inplace=True,drop=True)
    for row in df.index:
        if row>0:
            try:
                if df['player'][row]==df['player'][row-1]:
                    if df['minutes_90s'][row]>df['minutes_90s'][row-1]:#checking minutes per 90
                        df.drop(index=row-1,inplace=True)
                    else:
                        df.drop(index=row,inplace=True)                    
            except:
                try:
                    if df['minutes'][row]>df['minutes'][row-1]: #checking minutes
                        df.drop(index=row-1,inplace=True)
                    else:
                        df.drop(index=row,inplace=True)
                except:
                    continue
    df.reset_index(inplace=True,drop=True)
for df in df_fbref_list:
    del_id_pl(df)

In [5]:
pl_pass.loc[pl_pass['player']=='Theo Walcott']

,player,position,squad,minutes_90s,assisted_shots,passes_into_penalty_area,crosses_into_penalty_area
394,Theo Walcott,"FW,MF",Southampton,5.7,5,2,0


In [6]:
def del_null_fbref(df):#remove players who haven't played (fbref)
    for row in df.index:
        if df['games'][row]==0:
            df.drop(index=row,inplace=True)
    df.reset_index(inplace=True,drop=True)
del_null_fbref(pl_time)


In [7]:
pl_time.loc[pl_time['player']=='Diogo Jota']

,player,position,squad,games,minutes,minutes_per_game,minutes_pct,minutes_90s,games_starts,games_complete,...,minutes_per_sub,unused_subs,points_per_match,on_goals_for,on_goals_against,on_xg_for,on_xg_against,xg_plus_minus,xg_plus_minus_per90,xg_plus_minus_wowy
107,Diogo Jota,FW,Liverpool,9,503,56,50.8,5.6,5,3,...,16,0,2,12,11,11.5,10.2,1.3,0.23,-1.11


In [8]:
pd.options.mode.chained_assignment = None
def correct_team_names(teams_df):
    for row in teams_df.index: #changing team names in fantasy stats to use in fbref
        team_name=teams_df['name'][row]
        if team_name=='Leicester':
            teams_df['name'][row]='Leicester City'
        if team_name=='Man City':
            teams_df['name'][row]='Manchester City'
        if team_name=='Man Utd':
            teams_df['name'][row]='Manchester Utd'
        if team_name=='Newcastle':
            teams_df['name'][row]='Newcastle Utd'
        if team_name=='Spurs':
            teams_df['name'][row]='Tottenham'
        if team_name=='Leeds':
            teams_df['name'][row]='Leeds United'
correct_team_names(teams_df)
teams_df

,code,draw,form,id,loss,name,played,points,position,short_name,...,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,None,1,0,Arsenal,0,0,0,ARS,...,None,False,0,1190,1210,1170,1210,1190,1200,1
1,7,0,None,2,0,Aston Villa,0,0,0,AVL,...,None,False,0,1150,1160,1150,1150,1180,1210,2
2,36,0,None,3,0,Brighton,0,0,0,BHA,...,None,False,0,1080,1100,1150,1180,1090,1100,131
3,90,0,None,4,0,Burnley,0,0,0,BUR,...,None,False,0,1050,1080,1120,1190,1010,1030,43
4,8,0,None,5,0,Chelsea,0,0,0,CHE,...,None,False,0,1260,1280,1240,1280,1270,1310,4
5,31,0,None,6,0,Crystal Palace,0,0,0,CRY,...,None,False,0,1110,1150,1100,1150,1020,1050,6
6,11,0,None,7,0,Everton,0,0,0,EVE,...,None,False,0,1180,1210,1150,1170,1210,1250,7
7,54,0,None,8,0,Fulham,0,0,0,FUL,...,None,False,0,1000,1020,1020,1030,1020,1020,34
8,13,0,None,9,0,Leicester City,0,0,0,LEI,...,None,False,0,1220,1240,1190,1190,1200,1180,26
9,2,0,None,10,0,Leeds United,0,0,0,LEE,...,None,False,0,1100,1130,1060,1110,1130,1160,9


In [9]:
teams_list=[]
for team in teams_df['name']:
    teams_list.append(team)
teams_list

['Arsenal',
 'Aston Villa',
 'Brighton',
 'Burnley',
 'Chelsea',
 'Crystal Palace',
 'Everton',
 'Fulham',
 'Leicester City',
 'Leeds United',
 'Liverpool',
 'Manchester City',
 'Manchester Utd',
 'Newcastle Utd',
 'Sheffield Utd',
 'Southampton',
 'Tottenham',
 'West Brom',
 'West Ham',
 'Wolves']

In [10]:
def add_team_name(df):#adding team name
    df['team_name']='' 
    for row in df.index: 
        team_id=df['team'][row]
        team_row=teams_df.loc[teams_df['id']==team_id]
        team_row.index=[0]
        team_name=team_row['name'][0]
        df['team_name'][row]=team_name

In [11]:
def remove_null_players(df):# remove players who haven't played yet (fantasy)
    for row in df.index: 
        if df['minutes'][row]==0.:
            df.drop(index=row,inplace=True)
    df.reset_index(inplace=True,drop=True)
    return df
global players
players=remove_null_players(elements_df)
add_team_name(players)

In [12]:
def update_position(df):#Updating position slot
    for row in df.index: 
        pos=df['element_type'][row]
        if pos==1:
            df['element_type'][row]='GK'
        if pos==2:
            df['element_type'][row]='DF'
        if pos==3:
            df['element_type'][row]='MF'
        if pos==4:
            df['element_type'][row]='FW'
update_position(players)
players[['web_name','element_type']]

,web_name,element_type
0,David Luiz,DF
1,Aubameyang,MF
2,Lacazette,FW
3,Mustafi,DF
4,Leno,GK
...,...,...
428,Marçal,DF
429,Vitinha,MF
430,Hoever,DF
431,Semedo,DF


In [13]:
def add_full_name(df):#add full name to players
    df['name']=''
    for row in df.index:
        first_name=df['first_name'][row]
        web_name=df['web_name'][row]
        if first_name in web_name: #build a name for checking
            name=web_name
        else:
            name=first_name+" "+web_name
        df['name'][row]=name
add_full_name(players)

In [14]:
def correct_bad_names(df):
    df.loc[df['player']=='Jóhann Berg Guðmundsson','player']='Johann Berg Gudmundsson'
    df.loc[df['player']=='İlkay Gündoğan','player']='Ilkay Gündogan'
    df.loc[df['player']=='Carlos Vinícius','player']='Carlos Vinicius'
    df.loc[df['player']=='Łukasz Fabiański','player']='Lukasz Fabianski'
    df.loc[df['player']=='Tomáš Souček','player']='Tomas Soucek'
    df.loc[df['player']=='Dan Nlundulu','player']="Daniel N'Lundulu"

In [15]:
pl_df_list=[pl_goal,pl_msc,pl_pass_type,pl_pass,pl_time,pl_pos,pl_shot,pl_std]
def find_names(df,players):
    df['id']=0
    correct_bad_names(df)
    df_groups=df.groupby('squad')
    pl_groups=players.groupby('team_name')    
    for team in teams_list:
        fbref=df_groups.get_group(team)
        fantasy=pl_groups.get_group(team)
        for row in fantasy.index:
            checked=False
            player_id=fantasy['id'][row]
            team_name=fantasy['team_name'][row]
            name=fantasy['name'][row]
            first_name=fantasy['first_name'][row]
            second_name=fantasy['second_name'][row]
            web_name=fantasy['web_name'][row]
            pos=fantasy['element_type'][row]
            for s_row in fbref.index:
                fb_name=fbref['player'][s_row]              
                fb_team=fbref['squad'][s_row]
                fb_pos=fbref['position'][s_row]
                if name==fb_name or first_name+' '+second_name==fb_name: #find names which are totally equal
                    checked=True
                    df.loc[df['player']==fb_name,'id']=player_id
                    fantasy.drop(index=row,inplace=True)
                    fbref.drop(index=s_row,inplace=True)
                    break
            if checked==False: #find names which are not totally equal
                for s_row in fbref.index:
                    fb_name=fbref['player'][s_row]              
                    fb_team=fbref['squad'][s_row]
                    fb_pos=fbref['position'][s_row]
                    if web_name in fb_name or second_name in fb_name:
                        df.loc[df['player']==fb_name,'id']=player_id
                        checked=True
                    elif (first_name in fb_name or fb_name in first_name) and pos in fb_pos:
                        df.loc[df['player']==fb_name,'id']=player_id
                        checked=True
for df in pl_df_list:#finding names
    find_names(df,players)

In [16]:
pl_std.loc[pl_std['squad']=='Manchester City']

,player,position,squad,games,games_starts,minutes,goals,assists,pens_made,pens_att,...,goals_assists_pens_per90,xg,npxg,xa,xg_per90,xa_per90,xg_xa_per90,npxg_per90,npxg_xa_per90,id
42,Aymeric Laporte,DF,Manchester City,4,4,360,0,0,0,0,...,0.00,0.2,0.2,0.2,0.04,0.06,0.10,0.04,0.10,280
52,Benjamin Mendy,DF,Manchester City,5,5,430,1,0,0,0,...,0.21,0.1,0.1,0.0,0.02,0.01,0.02,0.02,0.02,274
55,Bernardo Silva,"MF,FW",Manchester City,6,4,374,0,0,0,0,...,0.00,0.3,0.3,0.1,0.07,0.03,0.09,0.07,0.09,281
115,Ederson,GK,Manchester City,10,10,900,0,0,0,0,...,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,278
124,Eric García,DF,Manchester City,3,2,201,0,0,0,0,...,0.00,0.0,0.0,0.1,0.01,0.04,0.04,0.01,0.04,288
135,Fernandinho,MF,Manchester City,5,2,201,0,0,0,0,...,0.00,0.5,0.5,0.0,0.20,0.00,0.20,0.20,0.20,266
137,Ferrán Torres,FW,Manchester City,7,5,397,1,1,0,0,...,0.45,1.9,1.9,0.2,0.43,0.04,0.47,0.43,0.47,287
143,Gabriel Jesus,FW,Manchester City,5,5,450,2,1,0,0,...,0.60,1.6,1.6,1.1,0.31,0.22,0.53,0.31,0.53,282
211,John Stones,DF,Manchester City,3,3,270,0,0,0,0,...,0.00,0.1,0.1,0.1,0.02,0.02,0.04,0.02,0.04,273
222,João Cancelo,"DF,MF",Manchester City,6,6,540,0,1,0,0,...,0.17,0.1,0.1,1.3,0.02,0.22,0.24,0.02,0.24,277


In [17]:
posts = players.groupby('element_type')

In [18]:
GK = posts.get_group('GK')
DF = posts.get_group('DF')
MF = posts.get_group('MF')
FW = posts.get_group('FW')

In [19]:
#find names for gk
gk_df_list=[pl_gk,pl_adv_gk] 
for df in gk_df_list:
    find_names(df,GK)
pl_adv_gk

,player,position,squad,minutes_90s,psxg_gk,psnpxg_per_shot_on_target_against,psxg_net_gk,psxg_net_per90_gk,id
0,Adrián,GK,Liverpool,2.0,7.1,0.45,-1.9,-0.93,245
1,Alisson,GK,Liverpool,8.0,8.7,0.35,0.7,0.09,252
2,Alphonse Areola,GK,Fulham,10.0,19.4,0.31,1.4,0.14,516
3,Kepa Arrizabalaga,GK,Chelsea,3.0,4.8,0.34,-1.2,-0.42,112
4,Willy Caballero,GK,Chelsea,1.0,1.5,0.49,-1.5,-1.54,101
5,Karl Darlow,GK,Newcastle Utd,10.0,14.6,0.24,0.6,0.06,325
6,Ederson,GK,Manchester City,10.0,9.8,0.27,-1.2,-0.12,278
7,Lukasz Fabianski,GK,West Ham,11.0,15.9,0.35,1.9,0.17,431
8,David de Gea,GK,Manchester Utd,8.5,14.0,0.40,-1.0,-0.12,291
9,Vicente Guaita,GK,Crystal Palace,11.0,16.1,0.29,0.1,0.01,128


In [20]:
pd.options.mode.chained_assignment = None 
def add_cl(post,df,cl): #add columns from teams_df to players
    post[cl] = 0 
    for row in post.index:
        team_id = post['team'][row] 
        team_row = df.loc[df['id'] == team_id]  
        post[cl][row] = team_row[cl]

In [21]:
stg_list=['strength_attack_home','strength_attack_away','strength_defence_home','strength_defence_away']
for stg in stg_list: #adding strength defence & attack to DF & MF
    add_cl(DF,teams_df,stg)
    add_cl(MF,teams_df,stg)
add_cl(FW,teams_df,stg_list[0])#add to GK
add_cl(FW,teams_df,stg_list[1])
add_cl(GK,teams_df,stg_list[2])#add to FW
add_cl(GK,teams_df,stg_list[3])

In [23]:
def add_cl_fbref(post,fbref,cl_list): #add columns from fbref to fantasy
    for cl in cl_list:
        post[cl]=0.0
    for row in post.index:
        for cl in cl_list:
            player_id=post['id'][row]
            fb_row=fbref.loc[fbref['id']==player_id]
            fb_row.index=[0]
            post[cl][row]=fb_row[cl][0]

In [24]:
fbref_cl=[[pl_std,std_cl],[pl_shot,shot_cl],[pl_pass,pass_cl],[pl_pass_type,pass_type_cl] 
    ,[pl_pos,pos_cl], [pl_goal,goal_cl], [pl_msc,msc_cl] ,  [pl_time,time_cl]] #fbref dataframes with desired columns

fbref_gk_cl=[[pl_gk,gk_cl],[pl_adv_gk,adv_gk_cl]]
post_list=[DF,MF,FW]
for post in post_list: #updating posts with fbref data
    for cl in fbref_cl:
        add_cl_fbref(post,cl[0],cl[1])
for cl in fbref_gk_cl:
    add_cl_fbref(GK,cl[0],cl[1])



In [25]:
%run columns.ipynb

In [ ]:
#MF.to_excel('C:\\Users\\farha\\MF.xlsx')
#FW.to_excel('C:\\Users\\farha\\FW.xlsx')
#DF.to_excel('C:\\Users\\farha\\DF.xlsx')
#GK.to_excel('C:\\Users\\farha\\GK.xlsx')

In [26]:
FW

,id,web_name,first_name,second_name,team,now_cost,cost_change_start,total_points,selected_by_percent,minutes,...,creativity_rank_type,threat_rank,threat_rank_type,transfers_in,transfers_out,ep_next,status,own_goals,team_code,name
0,6,Lacazette,Alexandre,Lacazette,1,83,-2,29,2.7,584,...,19,38,19,276846,421501,1.7,a,0,3,Alexandre Lacazette
1,22,Nketiah,Edward,Nketiah,1,55,-5,16,0.6,160,...,39,154,41,34992,128076,1.3,a,0,3,Edward Nketiah
2,50,Davis,Keinan,Davis,2,43,-2,1,7.4,28,...,46,325,50,524522,1055929,0.0,d,0,7,Keinan Davis
3,514,Watkins,Ollie,Watkins,2,61,1,49,8.1,810,...,4,15,8,1036125,551759,1.0,a,0,7,Ollie Watkins
4,68,Maupay,Neal,Maupay,3,62,-3,40,2.3,759,...,11,11,7,609914,619805,0.2,a,0,36,Neal Maupay
5,78,Connolly,Aaron,Connolly,3,53,-2,22,0.5,413,...,28,78,33,69409,82533,0.8,a,0,36,Aaron Connolly
6,588,Welbeck,Danny,Welbeck,3,55,0,22,0.9,362,...,25,62,26,74779,14471,5.8,a,0,36,Danny Welbeck
7,86,Rodriguez,Jay,Rodriguez,4,58,-2,13,0.3,407,...,36,138,40,10807,48958,1.7,a,0,90,Jay Rodriguez
8,87,Barnes,Ashley,Barnes,4,60,0,12,0.1,476,...,14,68,28,9045,13510,0.0,a,0,90,Ashley Barnes
9,91,Wood,Chris,Wood,4,62,-3,30,1.0,869,...,26,28,15,93676,186226,3.0,a,0,90,Chris Wood
